In [1]:
import torch
import os
import numpy as np
import pandas as pd
from biopandas.mol2 import PandasMol2
from biopandas.pdb import PandasPdb
from scipy.stats import special_ortho_group

import torch

In [2]:
# this next part is just for grabbing the plotting function that dave wrote
cwd = os.getcwd()
os.chdir('/Users/nisargjoshi/Desktop/direct_project/cnns4qspr/se3cnn_v2')
from dave_viz import plot_field
os.chdir('/Users/nisargjoshi/Desktop/direct_project/cnns4qspr/cnns4qspr/formatting_data')

In [21]:
def load_pdb(path):
    """
    This function will load all of the atomic positioning/type arrays 
    from a pdb file. This arrays can then be transformed into 
    density (or "field") tensors before being sent through the neural 
    network. 
    
    Parameters:
    ___________
    
    path (str, required): The full path to the pdb file being voxelized. 
    
    Returns: 
    ________
    
    protein_dict (dict): A dictionary containing the following arrays from 
                     the pdb file: n_atoms, atom_types, positions,
                     atom_type_set, ...
    """

    pdb = PandasPdb().read_pdb(path)

    
    # This just creates a dataframe from the pdb file using biopandas 
    #print('This is vars',vars(pdb))
    pdf = pdb.df['ATOM']
    
    # atomic coordinates
    x_coords = pdf['x_coord'].values
    y_coords = pdf['y_coord'].values
    z_coords = pdf['z_coord'].values
    
    # create an array containing tuples of x,y,z for every atom
    positions = []
    for i, x in enumerate(x_coords):
        position_tuple = (x_coords[i], y_coords[i], z_coords[i])
        positions.append(position_tuple)
    positions = np.array(positions)
    
    # names of all the atoms contained in the protein
    atom_types = pdf['atom_name'].values
    num_atoms = len(atom_types)
    atom_type_set = np.unique(atom_types)
    num_atom_types = len(atom_type_set)
    
    # residue names
    residue_names = pdf['residue_name'].values
    
    protein_dict = {'x_coords':x_coords, 'y_coords':y_coords, 'z_coords':z_coords, 'positions':positions, 
                    'atom_types':atom_types, 'num_atoms':num_atoms, 'atom_type_set':atom_type_set,
                    'num_atom_types':num_atom_types, 'residue_names':residue_names}
    
    
    return protein_dict

In [38]:
protein_dict = load_pdb('file/1a1vA02')

In [39]:
protein_dict['positions']
x_cond=[protein_dict['x_coords'].min(),protein_dict['x_coords'].max()]
y_cond=[protein_dict['y_coords'].min(),protein_dict['y_coords'].max()]
z_cond=[protein_dict['z_coords'].min(),protein_dict['z_coords'].max()]


if x_cond[1]>50:
    a=50
else:
    a=0

if y_cond[1]>50:
    b=50
else:
    b=0
    
if z_cond[1]>50:
    c=50
else:
    c=0
    
new_list=[a,b,c]

new_pos=protein_dict['positions']-new_list

In [40]:
def voxelize(protein_dict, bin_size=2.0, num_bins=50):
    """
    This function takes a protein dict (from load_pdb function) and outputs a large tensor containing many 
    atomic "fields" for the protein. The fields describe the atomic "density" (an exponentially decaying function 
    of number of atoms in a voxel) of any particular atom type. 
    
    Parameters:
    ___________
    protein_dict (dict, requred): dictionary from the load_pdb function
    
    Returns:
    ________
    fields (numpy array or pytorch tensor): A list of atomic density tensors (50x50x50)
    """
    
    # this is weird tuple construction going into this zeros call
    # basically if the protein had 27 atom types, and we use 50 bins, 
    # this ends up being zeros((37,50,50,50))
    fields = torch.zeros(*(protein_dict['num_atom_types'],) + (num_bins, num_bins, num_bins))
    
    # create linearly spaced grid (default is -49 to 49 in steps of 2)
    grid_1d = torch.linspace(start=-num_bins / 2 * bin_size + bin_size / 2, 
                             end=num_bins / 2 * bin_size - bin_size / 2, 
                             steps=num_bins)
#     print(grid_1d)
#     print(grid_1d.shape)
    
    # This makes three 3D meshgrids in for the x, y, and z positions
    # These cubes will be flattened and then used to normalize atomic positions in the middle of a datacube
    xx = grid_1d.view(-1, 1, 1).repeat(1, len(grid_1d), len(grid_1d))
    yy = grid_1d.view(1, -1, 1).repeat(len(grid_1d), 1, len(grid_1d))
    zz = grid_1d.view(1, 1, -1).repeat(len(grid_1d), len(grid_1d), 1)
    
    for atom_type_index, atom_type in enumerate(protein_dict['atom_type_set']): # i put 26 just to grab Oxy
        
        # Extract positions of only the current atom type 
        atom_positions = new_pos[protein_dict['atom_types'] == atom_type]
        atom_positions = torch.FloatTensor(atom_positions)
        
        
#         b = torch.tensor([1,2,3,4,5])
#         print(b)
#         print('*** repeat b')
#         rb = b.repeat(4,1)
#         print(rb)
        
        # xx.view(-1, 1) is 125,000 long, because it's viewing a 50x50x50 cube in one column
        # then you repeat that column horizontally for each atom
        xx_xx = xx.view(-1, 1).repeat(1, len(atom_positions))
        yy_yy = yy.view(-1, 1).repeat(1, len(atom_positions))
        zz_zz = zz.view(-1, 1).repeat(1, len(atom_positions))
        # at this point we've created 3 arrays that are 125,000 long 
        # and as wide as the number of atoms that are the current atom type
        # these 3 arrays just contain the flattened x,y,z positions of our 50x50x50 box
        
        
        # now do the same thing as above, just with the ACTUAL atomic position data
        posx_posx = atom_positions[:, 0].contiguous().view(1, -1).repeat(len(xx.view(-1)), 1)
        posy_posy = atom_positions[:, 1].contiguous().view(1, -1).repeat(len(yy.view(-1)), 1)
        posz_posz = atom_positions[:, 2].contiguous().view(1, -1).repeat(len(zz.view(-1)), 1)
        # three tensors of the same size, with actual atomic coordinates
        
        # normalizes the atomic positions with respect to the center of the box 
        # and calculates density of atoms in each voxel
        sigma = 0.5*bin_size
        density = torch.exp(-((xx_xx - posx_posx)**2 
                            + (yy_yy - posy_posy)**2 
                            + (zz_zz - posz_posz)**2) / (2 * (sigma)**2))
        
        print(torch.nonzero(density))
        print(len(torch.nonzero(density)))
        print('shape density is \n', density.shape)
        # Normalize so each atom density sums to one
        density /= torch.sum(density, dim=0)

        # Sum densities and reshape to original shape
        sum_densities = torch.sum(density, dim=1).view(xx.shape)
        print('sum densities shape is \n', sum_densities.shape)
        
        # set all nans to 0
        sum_densities[sum_densities != sum_densities] = 0
        
        fields[atom_type_index] = sum_densities
        
        if atom_type_index > 4:
            break
    
    return fields

        

In [41]:
fields = voxelize(protein_dict)

tensor([[ 56083,      8],
        [ 56083,      9],
        [ 56084,      8],
        ...,
        [124137,     58],
        [124138,     58],
        [124139,     58]])
211101
shape density is 
 torch.Size([125000, 135])
sum densities shape is 
 torch.Size([50, 50, 50])
tensor([[ 55983,      9],
        [ 55984,      9],
        [ 55985,      9],
        ...,
        [124089,     59],
        [124090,     58],
        [124090,     59]])
210993
shape density is 
 torch.Size([125000, 135])
sum densities shape is 
 torch.Size([50, 50, 50])
tensor([[ 53534,      8],
        [ 53535,      8],
        [ 53536,      8],
        ...,
        [124088,     54],
        [124089,     53],
        [124090,     53]])
188830
shape density is 
 torch.Size([125000, 121])
sum densities shape is 
 torch.Size([50, 50, 50])
tensor([[ 55933,      3],
        [ 55934,      3],
        [ 55935,      3],
        ...,
        [124033,     15],
        [124034,     15],
        [124035,     15]])
51293
shape de

In [42]:
plot_field(fields[0])

In [43]:
plot_field(fields[1])

In [45]:
plot_field(fields[2])

In [44]:
plot_field(fields[3])